# Civil Engineering Attendance History
This notebook is used to plot student attendance primarily on a module level. It draws data exported (via CSV/Excel) from Northumbia University Attendance Monitoring System.

In [ ]:
import datetime
import os

sem = 1
num_wks = 12
        
# Week numbers/dates
wks = []
wk_start = datetime.datetime(2022, 9, 26)
for i in range(1, 13):
    wk_end = wk_start + datetime.timedelta(days=4)
    wks.append((wk_start, wk_end))
    wk_start = wk_start + datetime.timedelta(days=7)
    print(f'Wk{i}: {wks[i - 1][0]:%d-%m-%Y} to {wks[i - 1][1]:%d-%m-%Y}')           
    

In [ ]:
import matplotlib as mpl
import matplotlib.pyplot as plt
from matplotlib.pyplot import cm
%matplotlib inline

import numpy as np

from openpyxl import load_workbook

class Module():
    def __init__(self, code):
        self.name = None
        self.code = code
        self.wks_with_events = [0] * num_wks
        self.attendance_possible = [0] * num_wks
        self.attendance_actual = [0] * num_wks

# Module info
modules = []

mod_names = ['KB4023', 'KB4024', 'KC4018', 'KB5021', 'KB5025', 'KL5002', 'KB6012', 'KB6022', 'KB6032', 'KB7012', 'KB7014', 'KB7034']
#mod_names = ['KB4022', 'KB4027', 'KB4028', 'KB5020', 'KB5022', 'KB5024', 'KB6031', 'KB6032', 'KB6048', 'KB7013', 'KB7015', 'KB7016']
for m in mod_names:
    modules.append(Module(code=m))
    
# Load attendance data
path_parent = '/Users/cwarren/Dropbox/Work/Teaching/Programme Leader/2022-2023/Attendance'
os.chdir(os.path.join(path_parent, 'sem' + str(sem)))
for wk in range(0, num_wks):
    fn = 'wk' + str(wk + 1)
    wb = load_workbook(filename = fn + '.xlsx')
    ws = wb[fn]

    for m in modules:
        event = False
        for row in ws.iter_rows(min_row=2, max_col=24, max_row=1000):
            if row[12].value == m.code:                   
                m.attendance_possible[wk] += row[10].value
                m.attendance_actual[wk] += row[8].value
                event = True
        if event:
            m.wks_with_events[wk] = 1

In [ ]:
# Plots
color = iter(cm.rainbow(np.linspace(0, 1, len(modules))))
ave_l4 = []
ave_l5 = []
ave_l6 = []
ave_l7 = []

fig, axs = plt.subplots(4, 3, figsize=(15, 12), facecolor='w', edgecolor='k')
axs = axs.ravel()

for mm, m in enumerate(modules):
    m.attendance_possible = np.array(m.attendance_possible)
    m.attendance_actual = np.array(m.attendance_actual)
    m.wks_with_events = np.array(m.wks_with_events)
    c = next(color)
    axs[mm].grid(True)
    x_label = [str(x) for x in range(1,num_wks + 1)]
    x_label[4] = 'A/W'
    x_pos = [i for i, _ in enumerate(x_label)]
    axs[mm].set_xticks(x_pos, x_label)
    axs[mm].set_ylim(0, 100)
    axs[mm].set_yticks(np.arange(0, 110, step=10))
    axs[mm].set_title(m.code, fontsize=16)
    
    wk_ave = np.divide(m.attendance_actual, m.attendance_possible)
    wk_ave = np.nan_to_num(wk_ave)
    
    axs[mm].bar(x_pos, wk_ave * 100, color=c)
    ave = (np.sum(wk_ave) / np.sum(m.wks_with_events)) * 100

    axs[mm].axhline(y=ave, linewidth=1, linestyle='--', color='k')
    ave = round(ave, 0)
    txt = str(ave) + '%'
    axs[mm].text(10, np.ceil(ave) + 5, txt, fontsize = 12)
    if m.code[2] == '4':
        ave_l4.append(ave)
    elif m.code[2] == '5':
        ave_l5.append(ave)
    elif m.code[2] == '6':
        ave_l6.append(ave)
    elif m.code[2] == '7':
        ave_l7.append(ave)

for ax in axs.flat:
    ax.set(xlabel='Week number', ylabel='Attendance [%]')
#     ax.label_outer()
fig.tight_layout()   
fig.savefig('Attendance.pdf', bbox_inches='tight')
    
# Level averages
ave_l4 = np.mean(np.array(ave_l4))
ave_l5 = np.mean(np.array(ave_l5))
ave_l6 = np.mean(np.array(ave_l6))
ave_l7 = np.mean(np.array(ave_l7))

print('Level averages...')
print(f'Level 4: {ave_l4:.1f}%')
print(f'Level 5: {ave_l5:.1f}%')
print(f'Level 6: {ave_l6:.1f}%')
print(f'Level 7: {ave_l7:.1f}%')